# Function 1

In [ ]:
Function 1: Searching for Contamination Sources
This may sound simple because you only have a two-dimensional input, however it is a very difficult problem.
It corresponds to trying to find the source of radiation in some square area.
However, you can only detect the radiation once you are very close to it, meaning most of the readings will
be zero. There are two sources, one is not too dangerous, so make sure you try to find both modes of the
function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_1/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_1/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)
print(Y)

In [ ]:
y_max = np.max(Y)
print("Max output: ", y_max, "which corresponds: ", X[np.where(Y == y_max)][0])

Plotting the inputs first

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(8)
plt.scatter(X[:,0],X[:,1],c=Y)
plt.colorbar()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel as W

kernel = C(0.1, (1e-3, 1e1)) * RBF(length_scale=0.5, length_scale_bounds=(1e-2, 1e0)) + W(noise_level=1e-4, noise_level_bounds=(1e-5, 1e-2))

gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)
gpr.fit(X, Y)

In [ ]:
from scipy.stats import norm
######Acquisition Function 1 - Upper Confidence Bound####################
x1 = np.linspace(X[np.where(Y == y_max)][0][0]-0.07, X[np.where(Y == y_max)][0][0]+0.07, 2000)
x2 = np.linspace(X[np.where(Y == y_max)][0][1]-0.07, X[np.where(Y == y_max)][0][1]+0.07, 2000)
X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        X_grid.append([x1[i], x2[j]])
X_grid = np.array(X_grid)

mean, std = gpr.predict(X_grid, return_std = True)

ucb = mean + 0.76 * std

idx_max = np.argmax(ucb)
UB_NextQuery = X_grid[idx_max]
print("UCB - Next Query: ", UB_NextQuery)

####### Acquisition Function 2 - Probability of Improvement ################
def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - y_max) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query: ", PI_NextQuery)

########## Acquisition Function 3 - Expected Improvement############
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)



# Function 2

In [ ]:
Function 2 - Optimising Noisy Models
Problem Description This corresponds to trying to optimise an unknown machine learning model. However, the initialization of the model is very important,
meaning your observations will be very noisy, and the problem might have a lot of local optima! You are trying to make the model’s log-likelihood as large as possible.

Acquisition Function found to be the most efficient over time: Expected Improvement with Knowledge Gradient

Kernel Idea (evolved over time from simple default kernel usage to parametrized kernel based on inputs/outputs and function context):

The Matérn kernel is a generalization of the RBF and the absolute exponential kernel, and it has an additional parameter that controls the smoothness of the resulting function.
This can be particularly useful in our case, where the function may not be very smooth due to the presence of many local optima. Gaussian Process Regression also assumes that your function
observations are noisy, so we might want to include a White Noise kernel in our model to account for measurement noise. This is especially important if our model’s log-likelihood evaluations
are noisy.

The Matern kernel is specified with a length_scale and nu (set to 1.5). WhiteKernel is used to model the nose. ConstantKernel is used as a scaling factor.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_2/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_2/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)


In [ ]:
print(Y)

In [ ]:
y_max = np.max(Y)
print("Max output: ", y_max, "which corresponds: ", X[np.where(Y == y_max)][0])

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(8)
plt.scatter(X[:, 0], X[:, 1], c = Y)
plt.colorbar()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel as W

kernel = C(1.0, (1e-3, 1e2)) * Matern(length_scale=0.5, length_scale_bounds=(1e-2, 1e1)) + W(noise_level=1, noise_level_bounds=(1e-5, 1e1))

gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)
gpr.fit(X, Y)


In [ ]:
from scipy.stats import norm
######## Acquisition Function 1 - Upper Confidence Bound ##########
x1 = np.linspace(X[np.where(Y == y_max)][0][0]-0.07, X[np.where(Y == y_max)][0][0]+0.07, 2000)
x2 = np.linspace(X[np.where(Y == y_max)][0][1]-0.07, X[np.where(Y == y_max)][0][1]+0.07, 2000)
X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        X_grid.append([x1[i], x2[j]])
X_grid = np.array(X_grid)

mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 0.86 * std

idx_max = np.argmax(ucb1)
UB_NextQuery = X_grid[idx_max]

print("UCB - Next Query - Idea 1: ", UB_NextQuery)

########## Acquisition Function 2 - Probability of Improvement###########################
def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - y_max) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query: ", PI_NextQuery)

################# Acquisition Function 3 - Expected Improvement##################
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

x1 = np.linspace(X[np.where(Y == y_max)][0][0]-0.1, X[np.where(Y == y_max)][0][0]+0.07, 1000)
x2 = np.linspace(X[np.where(Y == y_max)][0][1]-0.1, X[np.where(Y == y_max)][0][1]+0.07, 1000)
X_grid = np.array([[a, b] for a in x1 for b in x2])

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)

# Function 3

In [ ]:
Function 3 - Drug Discovery Problem
Problem Description In this example, you are doing drug discovery! You can select three compounds to create a drug, and receive a measurement of the people’s adverse reaction to the drug.
You want to make this as close as possible to zero. (hint: one of the variables may not cause any effects on the person).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_3/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_3/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
y_max = np.max(Y)
print("Max output: ", y_max, "which corresponds: ", X[np.where(Y == y_max)][0])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=Y)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
print(X)
print(Y)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel as W

kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=0.1, length_scale_bounds=(1e-3, 1e1), nu=2.5) + W(noise_level=0.1, noise_level_bounds=(1e-10, 1e-1))

gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)

gpr.fit(X, Y)

In [ ]:
##############Acquisition Function 1 - Upper Confidence Bound################
x1 = np.linspace(0.73, 0.80, 250)
x2 = np.linspace(0.57, 0.65, 250)
x3 = np.linspace(0.40, 0.48, 250)

X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        for k in range(len(x3)):
            X_grid.append([x1[i], x2[j], x3[k]])

X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 0.86 * std

idx_max = np.argmax(ucb1)
UB_NextQuery = X_grid[idx_max]
print("UCB - Next Query - Idea 1: ", UB_NextQuery)


###### Acquisition Function 2 - Probability of Improvement

from scipy.stats import norm

def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - y_max) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query - Idea 1: ", PI_NextQuery)


###### Acquisition Function 3 - Thompson Sampling
x1 = np.linspace(0.73, 0.80, 150)
x2 = np.linspace(0.57, 0.65, 150)
x3 = np.linspace(0.40, 0.48, 150)

X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        for k in range(len(x3)):
            X_grid.append([x1[i], x2[j], x3[k]])

def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

next_idx = np.argmax(thompson_values)
TS_NextQuery = X_grid[next_idx]

print("Thompson Sampling - Next Query: ", TS_NextQuery)

###### Acquisition Function 4 - Bayesian Expected Losses

def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - y_max) / sigma
    expected_loss = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss

expected_loss_values = [compute_expected_loss(x) for x in X_grid]


next_idx = np.argmin(expected_loss_values)
BL_NextQuery = X_grid[next_idx]

print("Bayesian Expected Loss - Next Query: ", BL_NextQuery)

######## Acquisition Function 5 - Expected Improvement
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)

# Function 4

In [ ]:
Function 4: Fast, but Inaccurate Modelling
This example is for a particular business relying heavily on online sales.
It can run very accurate calculations to figure out what is the optimal placement of their product
across warehouses. Unfortunately, the calculations are extremely expensive (computationally) to run,
so they can only do it once every two weeks. Instead, they propose using a machine learning model which
approximates the solution quickly (in a few hours). The model has four hyper-parameters you need to tune,
and the output corresponds to the difference between the expensive calculation, and the model.
Since you are modelling a dynamical system, expect a lot of local optima!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_4/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_4/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)
print(Y)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=0.1)

gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)

gpr.fit(X, Y)

In [ ]:
###### Acquisition Function 1 - Upper Confidence Bound##########

import itertools as it

x_range = np.linspace(0, 1, 50)

dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)

mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 0.90 * std

idx_max = np.argmax(ucb1)
UB_NextQuery = X_grid[idx_max]
print("UCB - Next Query: ", UB_NextQuery)

####### Acquisition Function 2 - Probability of Improvement########
from scipy.stats import norm

def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - y_max) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query: ", PI_NextQuery)

###### Acquisition Function - Expected Improvement#########
from scipy.stats import norm

x_range = np.linspace(0, 1, 40)

dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)


def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - y_max) / sigma
    ei = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)

# Function 5

In [ ]:
Function 5: Yield in a Chemical Reaction
This time you are trying to optimise another four-dimensional black-box.
It corresponds to the yield of a chemical process after processing in some factory.
This type of process tends to be unimodal.
Try to find the combination of chemicals that maximizes the yield!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_5/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_5/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)
print(Y)

In [ ]:
print(np.max(Y))

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C
from scipy.optimize import minimize

kernel_matern = C(1.0, (1e-3, 1e3)) * Matern(length_scale=1.0, nu=1.5)  # Adjust nu for smoothness

gpr = GaussianProcessRegressor(kernel=kernel_matern, n_restarts_optimizer=9)

gpr.fit(X, Y)

In [ ]:
######## Acquisition Function 1 - Upper Confidence Bound ########
import itertools as it

x_range = np.linspace(0, 1, 50)

dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)

mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 1.10 * std

idx_max = np.argmax(ucb1)
UB_NextQuery = X_grid[idx_max]
print("UCB - Next Query", UB_NextQuery)

######## Acquisition Function 2 - Probability of Improvement #########
f_best = np.max(Y)

def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query", PI_NextQuery)

####### Acquisition Function 3  - Expected Improvement ###################
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)

# Function 6

In [ ]:
Function 6: Cake and Stuff
Time to get cooking! You are optimising a cake recipe. There are five ingredients.
The outputs correspond to the sum of different objectives: flavor, consistency, calories, waste and cost.
Each objective receives negative points by our expert taster.
You want this sum to be as close to zero as possible!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_6/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_6/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)
print(Y)

In [ ]:
print(np.max(Y))
print(np.min(Y))

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[1.0, 0.5, 2.0, 0.8, 1.5])

gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)
gpr.fit(X, Y)

In [ ]:
###### Acquisition Function 1 - Upper Confidence Bound
import itertools as it

x1 = np.linspace(0, 1, 30)

dim = 5
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 1.56 * std

idx_max = np.argmax(ucb1)
next_query = X_grid[idx_max]
print("UCB - Next Query", next_query)

######### Acquisition Function 2 - Probability of Improvement#########
from scipy.stats import norm

f_best = np.max(Y)

def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
next_query = X_grid[next_idx]

print("PI - Next Query", next_query)

######### Acquisition Function 3 - Expected Improvement ##########
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement", next_query)

# Function 7


In [ ]:
Function 7: Sometimes Lazy is Best
You are now optimising six hyper-parameters of a machine learning model.
Note that it is a popular and frequently used model, so maybe you could search to see if anyone
else has optisized it before?

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_7/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_7/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)
print(Y)

In [ ]:
print(np.max(Y))
print(np.min(Y))

In [ ]:
y_max = np.max(Y)
print("Max output: ", y_max, "which corresponds to: ", X[np.where(Y == y_max)][0])

In [ ]:
#####Build a Bayesian Model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

kernel_matern = 1.0 * Matern(length_scale=1.0, nu=1.5)  # Adjust nu for smoothness
gpr = GaussianProcessRegressor(kernel=kernel_matern, n_restarts_optimizer=10)
gpr.fit(X, Y)

In [ ]:
######Acquisition Function 1 - Upper Confidence Bound
from scipy.stats import norm
import itertools as it

x1 = np.linspace(0, 1, 15)

list(it.product(['1','2','3','4',], ['a', 'b','c','d']))

dim = 6
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 1.16 * std

idx_max = np.argmax(ucb1)
next_query = X_grid[idx_max]
print("UCB - Next Query: ", next_query)

########Acquisition Function 2 - Probability of Improvement
from scipy.stats import norm

f_best = np.max(Y)

def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
next_query = X_grid[next_idx]

print("PI - Next Query", next_query)

######Acquisition Function 3 - Thompson Sampling
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

next_idx = np.argmax(thompson_values)
next_query = X_grid[next_idx]

print("Thompson Sampling", next_query)

#######Acquisition Function 4 - Bayesian Expected Losses
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - f_best) / sigma
    expected_loss = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss

expected_loss_values = [compute_expected_loss(x) for x in X_grid]

next_idx = np.argmin(expected_loss_values)
next_query = X_grid[next_idx]

print("Bayesian Expected Loss", next_query)

########Acquisition Function 5 - Expected Improvement
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement", next_query)

# Function 8

In [ ]:
Function 8: High-dimensional Optimisation
You’ve reach the final, 8-dimensional search space.
High-dimensional black-box optimisation can be very difficult, so sticking to local solutions is not
the worst idea here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize


file_inputs  = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_8/initial_inputs.npy'
file_outputs = '/content/drive/My Drive/Colab Notebooks/IMP-PCMLAI-capstone-initial_data/initial_data/function_8/initial_outputs.npy'

X = np.load(file_inputs)
Y = np.load(file_outputs)

In [ ]:
print(X)
print(Y)

In [ ]:
y_max = np.max(Y)
print("Max output: ", y_max, "which corresponds to: ", X[np.where(Y == y_max)][0])

In [ ]:
####### Build a Bayesian Model
# Create a Gaussian Process Regressor with the specified kernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel as W

# Adjust the kernel parameters to suit the scale of the problem
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=0.1, length_scale_bounds=(1e-3, 1e1), nu=2.5) + W(noise_level=0.1, noise_level_bounds=(1e-10, 1e-1))

# Create the Gaussian Process Regressor model
gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=10)

gpr.fit(X, Y)

In [ ]:
########Acquisition Function 1 - Upper Confidence Bound
import itertools as it

x1 = np.linspace(0, 1, 8)

dim = 8
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 0.96 * std

idx_max = np.argmax(ucb1)
next_query = X_grid[idx_max]
print("UCB - Next Query: ", next_query)

In [ ]:
######## Acquisition Function 2 - Probability of Improvement
from scipy.stats import norm

f_best = np.max(Y)

def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

next_idx = np.argmax(pi_values)
next_query = X_grid[next_idx]

print("PI - Next Query: ", next_query)

In [ ]:
#######Acquisition Function - Thompson Sampling
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

next_idx = np.argmax(thompson_values)
next_query = X_grid[next_idx]

print("Thompson Sampling - Next Query: ", next_query)

In [ ]:
######## Acquisition Function - Bayesian Expected Losses
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - f_best) / sigma
    expected_loss = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss


expected_loss_values = [compute_expected_loss(x) for x in X_grid]

next_idx = np.argmin(expected_loss_values)
next_query = X_grid[next_idx]

print("Bayesian Expected Loss - Next Query: ", next_query)

In [ ]:
####### Acquisition Function - Expected Improvement
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement - Next Query: ", next_query)